In [17]:
import torch
from torch import LongTensor
from torch.nn import Embedding, LSTM
from torch.autograd import Variable
from torch.nn.utils.rnn import pack_padded_sequence as pack
from torch.nn.utils.rnn import pad_packed_sequence as unpack

batch_size = 3인 세 문장에 대해 LSTM을 돌려보자.

[['안녕', '친구야','반가워','나는','소영'],
['친구야','반가워']
['나는','소영','친구야','안녕']]

Step 1. Vocabulary를 만든다  
Step 2. index 부여된 데이터를 불러온다. (이중 리스트, 내부 리스트는 토큰 인덱스 리스트이다)  
Step 3. 모델 만들기
Step 4. 가장 긴 시퀀스 길이를 기준으로 내부리스트를 0으로 패딩한다.  
Step 5. 내부 리스트를 시퀀스 길이 기준으로 내림차순 정렬한다.  
Step 6. 내부 리스트를 임베딩 한다.  
Step 7. pack_padded_sequences 를 호출한다. 이 때 임베딩된 instance와 시퀀스 길이를 넘긴다  
Step 8. LSTM forward  
Step 9. unpack_padded_sequences 를 호출하거나 가장 마지막 히든 벡터를 선택한다.  

---

In [18]:
# We want to run LSTM on a batch following 3 token sequences
seqs = [['안녕', '친구야','반가워','나는','소영'], #len = 5
        ['친구야','반가워'],# len = 2
        ['나는','소영','친구야','안녕']]    # len = 4

### Step 1. Vocabulary 만들기

In [19]:
# make sure <pad> idx is 0
vocab = ['<pad>'] + sorted(set([token for seq in seqs for token in seq]))

['<pad>', '나는', '반가워', '소영', '안녕', '친구야']

### Step 2. 인덱싱된 데이터 불러오기

In [20]:
## Step 2: Load indexed data (list of instances, where each instance is list of character indices) ##
##-------------------------------------------------------------------------------------------------##
vectorized_seqs = [[vocab.index(token) for token in seq]for seq in seqs]
vectorized_seqs

[[4, 5, 2, 1, 3], [5, 2], [1, 3, 5, 4]]

### Step 3. 모델 만들기

In [21]:
embed = Embedding(len(vocab), 4) # len(vocab) = 6, embedding_dim = 4
lstm = LSTM(input_size=4, hidden_size=5, batch_first=True) # input_dim = 4, hidden_dim = 5

In [33]:
len(vocab)

6

### Step 4. 0으로 패딩하기

In [24]:
# get the length of each seq in your batch
seq_lengths = LongTensor(list(map(len, vectorized_seqs)))
seq_lengths

tensor([5, 2, 4])

In [27]:
seq_tensor = Variable(torch.zeros((len(vectorized_seqs), seq_lengths.max()))).long()
seq_tensor

tensor([[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]])

In [34]:
for idx, (seq, seqlen) in enumerate(zip(vectorized_seqs, seq_lengths)):
    seq_tensor[idx, :seqlen] = LongTensor(seq)
seq_tensor

tensor([[4, 5, 2, 1, 3],
        [5, 2, 0, 0, 0],
        [1, 3, 5, 4, 0]])

In [32]:
seq_tensor.shape #(batch_size X max_seq_len)

torch.Size([3, 5])

### Step 5. 길이로 내림차순 정렬

In [36]:
seq_lengths, perm_idx = seq_lengths.sort(0, descending=True)
seq_tensor = seq_tensor[perm_idx]
seq_tensor
# seq_tensor => [[ 6  9  8  4  1 11 12 10]           # long_str
#                [ 7  3  2  5 13  7  0  0]           # medium
#                [12  5  8 14  0  0  0  0]]          # tiny
# seq_tensor.shape : (batch_size X max_seq_len) = (3 X 8)

tensor([[4, 5, 2, 1, 3],
        [1, 3, 5, 4, 0],
        [5, 2, 0, 0, 0]])

### Step 6. 임베딩 하기

In [38]:
embedded_seq_tensor = embed(seq_tensor)
embedded_seq_tensor

tensor([[[-1.1293, -0.5929, -0.6378, -0.4917],
         [-0.1655, -1.1134, -0.2155,  0.0796],
         [ 0.0635,  0.0710,  0.1764,  1.3981],
         [ 0.0888, -1.3879, -1.3143, -2.0325],
         [-0.8205,  0.9168,  2.1129, -0.3407]],

        [[ 0.0888, -1.3879, -1.3143, -2.0325],
         [-0.8205,  0.9168,  2.1129, -0.3407],
         [-0.1655, -1.1134, -0.2155,  0.0796],
         [-1.1293, -0.5929, -0.6378, -0.4917],
         [-0.7626, -0.9101, -0.1752, -1.0258]],

        [[-0.1655, -1.1134, -0.2155,  0.0796],
         [ 0.0635,  0.0710,  0.1764,  1.3981],
         [-0.7626, -0.9101, -0.1752, -1.0258],
         [-0.7626, -0.9101, -0.1752, -1.0258],
         [-0.7626, -0.9101, -0.1752, -1.0258]]], grad_fn=<EmbeddingBackward>)